## Tokenizer

In [7]:
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer(preserve_case=False)

In [8]:
import torch, torchtext
from torchtext.datasets import text_classification

In [9]:
device = torch.device("cuda:1")

## Data load

In [13]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np


f_s = "/home/bwlee/data/yelp_review_polarity_csv/reviews_with_splits_full.csv"
df = pd.read_csv(f_s, header=0)
label0 = []
for i in df.index: # change this for getting info based on previous day market move
    if df['rating'].iloc[i] == 'positive':
        label0.append(1)
    else:
        label0.append(0)
df['rating'] = label0

In [24]:
train_df = df[df['split']=='train'][['rating', 'review']]
val_df = df[df['split']=='val'][['rating', 'review']]
test_df = df[df['split']=='test'][['rating', 'review']]
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [29]:
train_df.head()

,rating,review
0,0,the entrance was the impressive thing about th...
1,0,"i m a mclover , and i had no problem nwith the..."
2,0,"less than good here , not terrible , but i see..."
3,0,i don t know if i can ever bring myself to go ...
4,0,food was ok good but the service was terrible ...


## Set data field

In [35]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [38]:
from torchtext import data

TEXT = data.Field(sequential=True, use_vocab=True,
                 tokenize='spacy', lower=True,
                 batch_first=True, fix_length=100
                 )

LABEL = data.Field(sequential=False, use_vocab=False,
                  batch_first=False, is_target=True)

## Make dataset

In [54]:
from torchtext.data import Dataset
from torchtext.data import Iterator

In [47]:
train_data = Dataset(train_df, fields=(
        ('label', LABEL), ('text', TEXT)))

In [56]:
train_loader = Iterator(dataset=train_data, batch_size=5)
batch = next(iter(train_loader))

KeyError: 126579

In [52]:
train_data